In [91]:
import pandas as pd
import os
import csv

# Configuración necesaria para que decouple detecte .env (donde están guardadas las configuraciones)
from decouple import config
from decouple import AutoConfig

config = AutoConfig(search_path=os.getcwd())

In [92]:
# Acá simplemente armamos una lista con todas las direcciones enteras de nuestros archivos csv
# Decidí armarlo en una función por si a la hora de utilizarlo se quisiera cambiar la dirección raíz
# Como predeterminado deje datasets que suele ser una dirección raíz común

def directoryLister(rootdir = 'datasets/'):

    filesDirs = []

    for rootdir, dirs, files in os.walk(rootdir):
        
        for file in files:
            
            filesDirs.append(os.path.join(rootdir, file))
        
    # print(filesDirs)
    return filesDirs

In [93]:
# Acá armamos una lista con todos los dataframes provenientes de los csv's

def dfListCreator(fileDirs):
    dataFrames = []
    for j in filesDirs:
        with open(j, 'r') as f:
            #d_reader = csv.DictReader(f)

            #get fieldnames from DictReader object and store in list
            #headers = d_reader.fieldnames
            
            dataFrames.append(pd.read_csv(j))
        f.close()
    return dataFrames
    # I had to use the column names, because of the bug that makes some rows one column larger


In [94]:
def column_checker(df,columns):
    for j in columns:
        if j not in df.columns:
            print('{} no está entre las columnas'.format(j))
            return False

In [95]:
# La función column_checker está diseñada para evitar que un dataset malo corte el proceso y permitir así que los datasets buenos sean procesados
# No la uso en este caso, por que no queda claro que espera Alkemy , y como solo tengo 3 datasets , terminaria perdiendo mucha información
# Para evitar esto hice a mano los casos necesarios. Pero en un caso real, probablemente descartaría los datasets rotos, especialmente si fueran muchos
# Dependiendo siempre de si puedo o no prescindir de la información que estos brindan
# De todas maneras desde .env se puede elegir usar el checker o no.



def columnSelector(dfList,columns, columns_alt = '[]'):
    for j in range(0,len(dfList)):
        dfList[j].columns = dfList[j].columns.str.lower()
        dfList[j].columns = dfList[j].columns.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

        if CHECKER == True:
            if column_checker(dfList[j],columns):
                dfList[j] = dfList[j][columns]

        else:
            if 'domicilio' in dfList[j].columns:
                dfList[j] = dfList[j][columns]

                dfList[j].rename(columns = {'idprovincia':'id_provincia',
                                            'iddepartamento': 'id_departamento',
                                            'categoria':'categoría',
                                            'cp':'código postal',
                                            'telefono':'número de teléfono',}, inplace = True) 

            elif 'direccion' in dfList[j].columns:
                dfList[j] = dfList[j][columns_alt]

                dfList[j].rename(columns = {'idprovincia':'id_provincia',
                                            'iddepartamento': 'id_departamento',
                                            'categoria':'categoría',
                                            'cp':'código postal',
                                            'telefono':'número de teléfono',
                                            'direccion':'domicilio'}, inplace = True)

    return dfList




In [96]:
def dataPipeline():
    
    DATASETS_ROOT_DIR = config('DATASETS_DIR')

    # Obtiene direcciones de mis archivos csv en base a mi rootdir
    fileDirs = directoryLister(DATASETS_ROOT_DIR)
    
    # Crea una lista con los dataframes en base a los csv
    dfList = dfListCreator(fileDirs)

    # Procesa cada dataframe para seleccionar ciertas columnas establecidas en la configuracion .env
    columns = config(columns)
    columns_alt = config(columns_alt)
    CHECKER = config('CHECKER')

    dfList = columnSelector(dfList, columns, columns_alt)

    print(dfList[0].head())

    

In [97]:
dataPipeline()

      cod_loc  id_provincia  id_departamento              categoría  \
0  70049060.0          70.0          70049.0  Bibliotecas Populares   
1   2000010.0           2.0           2000.0  Bibliotecas Populares   
2   2000010.0           2.0           2000.0  Bibliotecas Populares   
3   2000010.0           2.0           2000.0  Bibliotecas Populares   
4   2000010.0           2.0           2000.0  Bibliotecas Populares   

                         provincia               localidad  \
0                         San Juan                   Rodeo   
1  Ciudad Autónoma de Buenos Aires  Ciudad de Buenos Aires   
2  Ciudad Autónoma de Buenos Aires  Ciudad de Buenos Aires   
3  Ciudad Autónoma de Buenos Aires  Ciudad de Buenos Aires   
4  Ciudad Autónoma de Buenos Aires  Ciudad de Buenos Aires   

                                        nombre             domicilio  \
0         Biblioteca Popular Juan P. Garramuno         Santo Domingo   
1  Biblioteca Popular Helena Larroque de Roffo          